<a href="https://colab.research.google.com/github/MINJIN82/practice/blob/main/type2_ex(blank)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 출처 및 연결
- 본 데이터 셋은 직접 다운받아 코랩에 올려주세요!
- 방법1: [데이터 다운받기](https://www.dataq.or.kr/www/board/view.do?bbsKey=eyJiYnNhdHRyU2VxIjoxLCJiYnNTZXEiOjUwOTM0M30=&boardKind=notice)
- 방법2: 직접 게시글 찾기: https://www.dataq.or.kr/ - 공지사항 - 759번 제2회 빅데이터분석기사

- 다운로드 : **[Dataset] 작업형 제2유형.zip**
    - X_train.csv
    - y_train.csv
    - X_test.csv
- 위 데이터 또는 게시글이 삭제될 시 유사 데이터 셋을 수업노트에 올려놓겠습니다.

백화점 고객의 1년 간 구매 데이터
- cust_id: 고객 ID
- gender: 고객의 성별 (0:여자, 1: 남자)
- 고객의 성별 예측값 (남자일 확률)을 다음과 같은 형식의 CSV 파일로 생성하시오



```
cust_id,gender
3500,0.267
3501,0.578
3502,0.123

```




성능이 우수한 예측 모형을 위해서는 아래 내용이 수반되어야 함
- 데이터 전처리
- Feature Engineering
- 모델링 (분류 알고리즘 사용)
- 하이퍼파라미터 튜닝 (초매개변수 최적화)
- csv제출 (확률 값)

유의사항
- 수험번호.csv 파일이 만들어지도록 코드를 제출함
- 제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점함

## 라이브러리 및 데이터 불러오기

In [ ]:
import pandas as pd

# 시험환경에서는 encoding="euc-kr"가 없어도 됨
X_train = pd.read_csv("X_train.csv", encoding="euc-kr")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv", encoding="euc-kr")

## EDA

In [ ]:
# 데이터 크기
X_train.shape, y_train.shape, X_test.shape

((3500, 10), (3500, 2), (2482, 10))

In [ ]:
# 데이터 샘플 X_train
X_train.head()
X_train.columns

Index(['cust_id', '총구매액', '최대구매액', '환불금액', '주구매상품', '주구매지점', '내점일수', '내점당구매건수',
       '주말방문비율', '구매주기'],
      dtype='object')

In [ ]:
# 데이터 샘플 y_train
y_train.head()

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0


In [ ]:
# 데이터 샘플 X_test
X_test.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,3500,70900400,22000000,4050000.0,골프,부산본점,13,1.461538,0.789474,26
1,3501,310533100,38558000,48034700.0,농산물,잠실점,90,2.433333,0.369863,3
2,3502,305264140,14825000,30521000.0,가공식품,본 점,101,14.623762,0.083277,3
3,3503,7594080,5225000,NaN,주방용품,부산본점,5,2.000000,0.000000,47
4,3504,1795790,1411200,NaN,수산품,청량리점,3,2.666667,0.125000,8


In [ ]:
# [Tip] 컬럼이 다보이지 않을 때
pd.set_option('display.Max_columns',None)

In [ ]:
# 결측치 확인 train
X_train.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [ ]:
# 결측치 확인 test
X_test.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       1611
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [ ]:
# 기초 통계
X_train.describe()

,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기
count,3500.000000,3.500000e+03,3.500000e+03,1.205000e+03,3500.000000,3500.000000,3500.000000,3500.000000
mean,1749.500000,9.191925e+07,1.966424e+07,2.407822e+07,19.253714,2.834963,0.307246,20.958286
std,1010.507298,1.635065e+08,3.199235e+07,4.746453e+07,27.174942,1.912368,0.289752,24.748682
min,0.000000,-5.242152e+07,-2.992000e+06,5.600000e+03,1.000000,1.000000,0.000000,0.000000
25%,874.750000,4.747050e+06,2.875000e+06,2.259000e+06,2.000000,1.666667,0.027291,4.000000
50%,1749.500000,2.822270e+07,9.837000e+06,7.392000e+06,8.000000,2.333333,0.256410,13.000000
75%,2624.250000,1.065079e+08,2.296250e+07,2.412000e+07,25.000000,3.375000,0.448980,28.000000
max,3499.000000,2.323180e+09,7.066290e+08,5.637530e+08,285.000000,22.083333,1.000000,166.000000


In [ ]:
# 기초 통계 object
X_train.describe(include = "object")

,주구매상품,주구매지점
count,3500,3500
unique,42,24
top,기타,본 점
freq,595,1077


In [ ]:
# 기초 통계 object (x_test)
X_test.describe(include = "object")

,주구매상품,주구매지점
count,2482,2482
unique,41,24
top,기타,본 점
freq,465,726


In [ ]:
# 고유값 (train)_'주구매상품'
X_train['주구매상품'].unique()

array(['기타', '스포츠', '남성 캐주얼', '보석', '디자이너', '시티웨어', '명품', '농산물', '화장품',
       '골프', '구두', '가공식품', '수산품', '아동', '차/커피', '캐주얼', '섬유잡화', '육류',
       '축산가공', '젓갈/반찬', '액세서리', '피혁잡화', '일용잡화', '주방가전', '주방용품', '건강식품',
       '가구', '주류', '모피/피혁', '남성 트랜디', '셔츠', '남성정장', '생활잡화', '트래디셔널',
       '란제리/내의', '커리어', '침구/수예', '대형가전', '통신/컴퓨터', '식기', '소형가전', '악기'],
      dtype=object)

In [ ]:
# 고유값 (test)
X_test['주구매상품'].unique()

array(['골프', '농산물', '가공식품', '주방용품', '수산품', '화장품', '기타', '스포츠', '디자이너',
       '시티웨어', '구두', '캐주얼', '명품', '건강식품', '남성정장', '커리어', '남성 캐주얼', '축산가공',
       '식기', '피혁잡화', '모피/피혁', '섬유잡화', '트래디셔널', '차/커피', '육류', '가구', '아동',
       '셔츠', '액세서리', '젓갈/반찬', '대형가전', '일용잡화', '통신/컴퓨터', '생활잡화', '주방가전',
       '란제리/내의', '남성 트랜디', '보석', '주류', '침구/수예', '악기'], dtype=object)

In [ ]:
# [Tip] set 타입으로 변경하면 비교 가능함
a=set(X_train['주구매상품'].unique())
b=set(X_test['주구매상품'].unique())

print(a.difference(b))

{'소형가전'}


In [ ]:
# target(label) 값 확인
y_train['gender'].value_counts()

0    2184
1    1316
Name: gender, dtype: int64

## 데이터 전처리

In [ ]:
# 결측치처리
X_train.isnull().sum()
X_train = X_train.fillna(0)
X_test.isnull().sum()
X_test = X_test.fillna(0)

In [ ]:
X_train

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,0.0,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,0.0,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,0.0,보석,본 점,2,1.500000,0.000000,85
...,...,...,...,...,...,...,...,...,...,...
3495,3495,3175200,3042900,0.0,골프,본 점,1,2.000000,1.000000,0
3496,3496,29628600,7200000,6049600.0,시티웨어,부산본점,8,1.625000,0.461538,40
3497,3497,75000,75000,0.0,주방용품,창원점,1,1.000000,0.000000,0
3498,3498,1875000,1000000,0.0,화장품,본 점,2,1.000000,0.000000,39


In [ ]:
# id 삭제함 (단 test의 id값은 csv파일을 생성할 때 필요함으로 옮겨 놓음)
X_train = X_train.drop('cust_id', axis =1)
cust_id = X_test.pop('cust_id')

## 피처엔지니어링

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총구매액     3500 non-null   int64  
 1   최대구매액    3500 non-null   int64  
 2   환불금액     3500 non-null   float64
 3   주구매상품    3500 non-null   object 
 4   주구매지점    3500 non-null   object 
 5   내점일수     3500 non-null   int64  
 6   내점당구매건수  3500 non-null   float64
 7   주말방문비율   3500 non-null   float64
 8   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 246.2+ KB


In [ ]:
# 라벨 인코더
from sklearn.preprocessing import LabelEncoder

cols = ['주구매상품','주구매지점']

for col in cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

X_train.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,6860000.0,5,0,19,3.894737,0.527027,17
1,2136000,2136000,300000.0,21,19,2,1.500000,0.000000,1
2,3197000,1639000,0.0,6,1,2,2.000000,0.000000,1
3,16077620,4935000,0.0,5,2,18,2.444444,0.318182,16
4,29050000,24000000,0.0,15,8,2,1.500000,0.000000,85


In [ ]:
# MinMaxScaler
# cols = X_train.select_dtypes(exclude = 'object').columns
# cols

cols =['총구매액', '최대구매액', '환불금액', '주구매상품', '주구매지점', '내점일수', '내점당구매건수', '주말방문비율','구매주기']

from sklearn.preprocessing import MinMaxScaler


# ms = MinMaxScaler()
# X_train[cols] = ms.fit_transform(X_train[cols])
# X_test[cols] = ms.transform(X_test[cols])

scaler = MinMaxScaler()
X_train[cols] = scaler.fit_transform(X_train[cols])
X_test[cols] = scaler.transform(X_test[cols])


X_train.head()



,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0.050810,0.020090,0.012168,0.121951,0.000000,0.063380,0.137300,0.527027,0.102410
1,0.022966,0.007226,0.000532,0.512195,0.826087,0.003521,0.023715,0.000000,0.006024
2,0.023412,0.006526,0.000000,0.146341,0.043478,0.003521,0.047431,0.000000,0.006024
3,0.028834,0.011171,0.000000,0.121951,0.086957,0.059859,0.068511,0.318182,0.096386
4,0.034295,0.038037,0.000000,0.365854,0.347826,0.003521,0.023715,0.000000,0.512048


## 데이터 분리

In [ ]:
# 검증 데이터 분리

from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train,
                                            y_train['gender'],
                                            test_size=0.2,
                                            random_state = 2022)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((2800, 9), (700, 9), (2800,), (700,))

## 모델링

In [ ]:
import sklearn.ensemble
print(sklearn.ensemble.__all__)

['BaseEnsemble', 'RandomForestClassifier', 'RandomForestRegressor', 'RandomTreesEmbedding', 'ExtraTreesClassifier', 'ExtraTreesRegressor', 'BaggingClassifier', 'BaggingRegressor', 'IsolationForest', 'GradientBoostingClassifier', 'GradientBoostingRegressor', 'AdaBoostClassifier', 'AdaBoostRegressor', 'VotingClassifier', 'VotingRegressor', 'StackingClassifier', 'StackingRegressor', 'HistGradientBoostingClassifier', 'HistGradientBoostingRegressor']


In [ ]:
# 모델링 & 하이퍼파라미터 튜닝 & 앙상블
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_tr,y_tr)
pred = model.predict_proba(X_val)

In [ ]:
# 예측 결과 확인
pred


array([[0.63, 0.37],
       [0.33, 0.67],
       [0.85, 0.15],
       ...,
       [0.49, 0.51],
       [0.6 , 0.4 ],
       [0.42, 0.58]])

## 평가(self)

In [ ]:
# import sklearn.metrics
# print(sklearn.metrics.__all__)

from sklearn.metrics import roc_auc_score
help(roc_auc_score)

In [ ]:
# 검증 데이터 셋으로 평가
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred[:, 1])


# X, y = load_breast_cancer(return_X_y=True)
# clf = LogisticRegression(solver="liblinear", random_state=0).fit(X, y)
# roc_auc_score(y, clf.predict_proba(X)[:, 1])

0.6222489980652295

## 예측 및 제출

In [ ]:
# test 데이터 예측
pred = model.predict_proba(X_test)
pred

array([[0.52, 0.48],
       [0.88, 0.12],
       [0.86, 0.14],
       ...,
       [0.48, 0.52],
       [0.46, 0.54],
       [0.36, 0.64]])

In [ ]:
# 데이터 프레임 만들기
sumit = pd.DataFrame(
          {"cust_id" : cust_id,
           "gender" : pred[:,1]
})

In [ ]:
# 샘플 데이터 확인
sumit.head()

,cust_id,gender
0,3500,0.48
1,3501,0.12
2,3502,0.14
3,3503,0.50
4,3504,0.30


In [ ]:
# csv 생성
sumit.to_csv("1111.csv",index=False)

##  제대로 제출 되었는지 확인

In [ ]:
pd.read_csv("1111.csv")

,cust_id,gender
0,3500,0.48
1,3501,0.12
2,3502,0.14
3,3503,0.50
4,3504,0.30
...,...,...
2477,5977,0.70
2478,5978,0.39
2479,5979,0.52
2480,5980,0.54
